# Score : 

#### Abandoned, not making much progress now.  Going to check the Kaggle discussion section for improvements

In [1]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Imputer
from xgboost import XGBRegressor

import pandas as pd

master_training_data = pd.read_csv('../data/house_prices/train.csv')
master_testing_data = pd.read_csv('../data/house_prices/test.csv')

In [2]:
training_data = master_training_data
testing_data = master_testing_data

# Shortcut to modify both at the same time
full_data = [training_data, testing_data]

#### Understanding (and modifying) SaleCondition

In [3]:
sale_cond_map = {"AdjLand" : 0, "Abnorml": 0, "Family": 0, "Alloca": 0, "Normal": 1, "Partial": 2}
for dataset in full_data:
    dataset['SaleCondition'] = dataset['SaleCondition'].map(sale_cond_map)

In [10]:
# There's definitely a better way of doing this
print(training_data.loc[training_data['SaleCondition'] == "AdjLand"].groupby("SaleCondition")['SalePrice'].transform("mean").head(2))

39     104125
386    104125
Name: SalePrice, dtype: int64
3    146526.623762
8    146526.623762
Name: SalePrice, dtype: float64
137    149600
154    149600
Name: SalePrice, dtype: int64
102    167377.416667
188    167377.416667
Name: SalePrice, dtype: float64
0    175202.219533
1    175202.219533
Name: SalePrice, dtype: float64
11    272291.752
13    272291.752
Name: SalePrice, dtype: float64


#### Sale Type

In [5]:
sale_type_map = {"WD": 0, "New": 1, "COD": 2, "ConLD": 2, "ConLw": 2, "ConLI": 2, "CWD": 2, "Oth": 2,"Con": 2}

for dataset in full_data:
    dataset['SaleType'] = dataset['SaleType'].map(sale_type_map)

#### Making the numbers in year sold smaller (does this do anything, I just feel compelled to do it)

In [6]:
for dataset in full_data:
    dataset['YrSold'] = dataset['YrSold'] - 2006

#### MSZoning: The general zoning classification

In [7]:
ms_zone_map = {"RL": 0, "RM": 1, "FV": 2, "RH": 2, "C (all)": 2}

for dataset in full_data:
    dataset['MSZoning'] = dataset['MSZoning'].map(ms_zone_map)

#### Alley: Type of alley access

In [8]:
ms_zone_map = {"Grvl": 0, "Pave": 2}

for dataset in full_data:
    dataset['Alley'] = dataset['Alley'].map(ms_zone_map)

In [9]:
training_data["Alley"].fillna(1, inplace=True)
testing_data["Alley"].fillna(1, inplace=True)

#### LotShape

In [10]:
lot_shape_map = {"Reg": 0, "IR1": 1, "IR2": 2, "IR3": 2}

for dataset in full_data:
    dataset['LotShape'] = dataset['LotShape'].map(lot_shape_map)

#### LandContour

In [11]:
lot_contour_map = {"Lvl": 0, "Bnk": 1, "HLS": 2, "Low": 3}

for dataset in full_data:
    dataset['LandContour'] = dataset['LandContour'].map(lot_contour_map)

#### LotConfig: Lot configuration

In [12]:
lot_config_map = {"Inside": 0, "Corner": 1, "CulDSac": 2, "FR2": 3, "FR3": 3}

for dataset in full_data:
    dataset['LotConfig'] = dataset['LotConfig'].map(lot_config_map)

#### LandSlope: Slope of property

In [13]:
land_slope_map = {"Gtl": 0, "Mod": 1, "Sev": 2}

for dataset in full_data:
    dataset['LandSlope'] = dataset['LandSlope'].map(land_slope_map)

#### Neighborhood: Physical locations within Ames city limits

### FIXME: This can be easily improved by looking at a map and seeing what neighboorhoods belong with each other

In [14]:
# This can be done A LOT better if I looked at a map!
neighborhood_map = {"NAmes": 0, "CollgCr": 1, "OldTown": 2, "Edwards": 3, "Somerst": 4,
                 "Gilbert": 5, "NridgHt": 6, "Sawyer": 7, "NWAmes": 0, "SawyerW": 7,
                  "BrkSide": 8, "Crawfor": 9, "Mitchel": 10, "NoRidge": 10, "Timber": 10,
                  "IDOTRR": 10, "ClearCr": 10, "SWISU": 10, "StoneBr": 10, "Blmngtn": 10,
                "MeadowV": 10, "BrDale": 10, "Veenker": 10, "NPkVill": 10, "Blueste": 10,
                 }

for dataset in full_data:
    dataset['Neighborhood'] = dataset['Neighborhood'].map(neighborhood_map)

#### Condition1: Proximity to main road or railroad

In [15]:
condition1_map = {"Norm": 0, "Feedr": 1, "Artery": 1, "RRAn": 1, "PosN": 1,
                 "RRAe": 1, "PosA": 1, "RRNn": 1, "RRNe": 1 }

for dataset in full_data:
    dataset['Condition1'] = dataset['Condition1'].map(condition1_map)

#### BldgType: Type of dwelling

In [16]:
bldg_type_map = {"1Fam": 0, "TwnhsE": 1, "Duplex": 2, "Twnhs": 2, "2fmCon": 2}

for dataset in full_data:
    dataset['BldgType'] = dataset['BldgType'].map(bldg_type_map)

#### HouseStyle: Style of dwelling

In [17]:
house_style_map = {"1Story": 0, "2Story": 1, "1.5Fin": 2, "SLvl": 3, "SFoyer": 3,
                "1.5Unf": 3, "2.5Unf": 3, "2.5Fin": 3}

for dataset in full_data:
    dataset['HouseStyle'] = dataset['HouseStyle'].map(house_style_map)

#### Fence: Fence quality

In [18]:
fence_map = {"MnPrv": 1, "GdPrv": 2, "GdWo": 3, "MnWw": 0}

for dataset in full_data:
    dataset['Fence'] = dataset['Fence'].map(fence_map)
    
training_data["Fence"].fillna(0, inplace=True)
testing_data["Fence"].fillna(0, inplace=True)

#### Electrical: Electrical system

In [19]:
electrical_map = {"SBrkr": 1, "FuseA": 2, "FuseF": 0, "FuseP": 0, "Mix": 0}

for dataset in full_data:
    dataset['Electrical'] = dataset['Electrical'].map(electrical_map)

#### CentralAir: Central air conditioning

In [20]:
central_air_map = {"N": 0, "Y": 1}

for dataset in full_data:
    dataset['CentralAir'] = dataset['CentralAir'].map(central_air_map)

#### BsmtFinType1: Quality of basement finished area

In [21]:
bsmt_fin_typ1_map = {"Unf": 0, "LwQ": 1, "Rec": 2, "BLQ": 3, "ALQ": 4, "GLQ": 5}

for dataset in full_data:
    dataset['BsmtFinType1'] = dataset['BsmtFinType1'].map(bsmt_fin_typ1_map)

In [22]:
qual_map = {"Po": 0, "Fa": 1, "TA": 2, "Gd": 3, "Ex": 4}

qual_cols = ['BsmtQual', 'BsmtCond', 'GarageCond', 'GarageQual', 'HeatingQC', 'KitchenQual']

for dataset in full_data:
    for col in qual_cols:
        dataset[col] = dataset[col].map(qual_map)

#### Functional: Home functionality (Assume typical unless deductions are warranted)

In [23]:
functional_map = {"Typ": 0, "Min1": 1, "Min2": 1, "Mod": 2, "Maj1": 3, "Maj2": 3, 'Sev': 4}

for dataset in full_data:
    dataset['Functional'] = dataset['Functional'].map(functional_map)

#### BsmtExposure: Refers to walkout or garden level walls

In [24]:
bsmt_exposure_map = {"No": 0, "Mn": 1, "Av": 2, "Gd": 3}

for dataset in full_data:
    dataset['BsmtExposure'] = dataset['BsmtExposure'].map(bsmt_exposure_map)

### Predictions!!

In [25]:
# ignoring street, Utilities, Condition2

predictor_cols = ['1stFlrSF','2ndFlrSF', 'Alley', 'BedroomAbvGr', 'BldgType', 'BsmtCond', 'BsmtFinType1',
                  'BsmtQual', 'CentralAir', 'Condition1', 'Electrical',
                  'Fence', 'Fireplaces', 'FullBath', 'GarageArea',
                 'GarageCars', 'GarageQual', 'GrLivArea', 'HalfBath', 'HeatingQC', 
                  'HouseStyle', 'KitchenAbvGr',
                  'LandSlope', 'LotArea', 'LotConfig',
                 'LotFrontage', 'LotShape', 'LowQualFinSF', 'MiscVal', 'MSSubClass', 'MSZoning', 'MoSold', 'Neighborhood', 'OpenPorchSF',
                 'OverallCond', 'OverallQual', 'SaleCondition', 'SaleType', 'ScreenPorch', 'TotRmsAbvGrd', 
                  'TotalBsmtSF', 'WoodDeckSF', 'YearBuilt', 'YearRemodAdd', 'YrSold']

train_x = training_data[predictor_cols]
train_y = training_data.SalePrice
test_x = testing_data[predictor_cols]

len(predictor_cols)

45

### Cross Validation scores

Using r2
XGBoost
- 18 predictors = 0.7702056269772473
- 24 predictors = 0.8018358449730526
- 27 predictors = 0.8299348333411789
- 29 predictors = 0.8479059750863349
- 38 predictors = 0.8685503276750433
- 41 predictors = 0.8883098714862785

In [26]:
k_fold = KFold(n_splits=10, shuffle=True, random_state=0)

pipeline = make_pipeline(Imputer(), XGBRegressor())

scoring = 'accuracy'
score = cross_val_score(pipeline, train_x, train_y, cv=k_fold, n_jobs=1, scoring="r2")
print(score)
print("\nAverage is ...")
print(sum(score) / len(score))

[0.88687809 0.87913453 0.9068024  0.89389989 0.79648544 0.9067508
 0.90139583 0.91060863 0.91411462 0.89434769]

Average is ...
0.8890417919221552


In [101]:
pipeline.fit(train_x, train_y)
prediction = pipeline.predict(test_x)

submission = pd.DataFrame({
        "Id": testing_data["Id"],
        "SalePrice": prediction
    })

submission.to_csv('/Users/pbezuhov/git/Kaggle/submissions/9_tuning_params.csv', index=False)